In [1]:
import pandas as pd 
import matplotlib.pyplot as plt
import datetime as dt
from sklearn.ensemble import IsolationForest
import numpy as np

In [2]:
df=pd.read_csv("rfm-simple-data2.csv")
df

,OrderNo,CustomerCode,SalesmanCode,OrderDate,ConfirmDate,ProductCode,Name,ProductManufacturer,StockQuantity,Group1,Group2,Quantity,NetPrice,NetAmount
0,1021047,120-6652,-1,30/12/2022 13:09:55,2/1/2023 10:57:27,EXQ 183001,YAKIT HORTUMU SPRINTER 06->09,EXQ,0,GENEL,GENEL,2,219.67,439.33
1,1021074,120-8813,-1,30/12/2022 14:00:28,2/1/2023 09:12:21,VM 963CPL,AYNA KAPAGI SOL ASTARLI INSIGNIA,VIEWMAX,0,KAPORTA,DIŞ AKSESUARLAR,1,127.06,127.06
2,1021136,120-4516,-1,30/12/2022 15:36:11,2/1/2023 09:12:13,TOPRAN 108033756,YAG DOLUM AGZI GOLF4-BORA-POLO 1.6 AEH-AKL 98-04,TOPRAN,0,MEKANİK,MOTOR,10,14.70,147.05
3,1021162,120-2082,-1,30/12/2022 16:23:52,2/1/2023 09:11:57,ROOT 111112,1313000 SIS KAPAK TAPASI UST SOL BUYUK ASTRA J,ROOT,28,KAPORTA,DIŞ AKSESUARLAR,1,30.67,30.67
4,1021163,120-8106,-1,30/12/2022 16:28:28,2/1/2023 09:11:53,AYD 01949,ROT BASI SAG T5-T6 04->,AYD,158,MEKANİK,ÖN-ARKA DÜZEN,2,121.27,242.54
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
392663,1190872,120-1037,-1,8/3/2024 13:02:33,8/3/2024 13:02:51,KALE 346260,KALORIFER RADYATORU POLO-FABIA-IBIZA-CORDOBA 1...,KALE,9,MEKANİK,ISITMA,1,882.87,882.87
392664,1190873,120-6836,-1,8/3/2024 13:11:12,8/3/2024 13:11:23,ESTAS 40055,EKSANTRIK MILI EKZOZ DW10BTED4 2.0 HDI 307 308...,ESTAS,6,MEKANİK,MOTOR,2,1862.70,3725.39
392665,1190874,120-7124,-1,8/3/2024 13:12:16,8/3/2024 13:12:40,AYD 06991,SALINCAK SOL DUCATO JUMPER III BOXER III 06->,AYD,177,MEKANİK,ÖN-ARKA DÜZEN,2,970.50,1941.00
392666,1190874,120-7124,-1,8/3/2024 13:12:16,8/3/2024 13:12:40,CAVO 1202005,EL FREN TELI SERCE 85002893 M124,CAVO,19,MEKANİK,FREN,5,247.23,1236.15


In [3]:
df['OrderDate'] = pd.to_datetime(df['OrderDate']).dt.date
ddf = df.groupby(['CustomerCode', 'OrderDate', 'ProductCode','Name'])['Quantity'].sum().reset_index()


C:\Users\ng822\AppData\Local\Temp\ipykernel_27208\2816129051.py:1: UserWarning: Parsing dates in %d/%m/%Y %H:%M:%S format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df['OrderDate'] = pd.to_datetime(df['OrderDate']).dt.date


In [4]:
customer_codes, customer_mapping = pd.factorize(ddf["CustomerCode"])
product_codes, product_mapping = pd.factorize(ddf["ProductCode"])
ddf["CustomerCode_"] = customer_codes
ddf["ProductCode_"] = product_codes

In [5]:
ddf

,CustomerCode,OrderDate,ProductCode,Name,Quantity,CustomerCode_,ProductCode_
0,120-0000,2023-01-02,APETECH AP1928,FREN BALATASI ARKA TEK TEKERLEK 2.3 M9T MASTER...,1,0,0
1,120-0000,2023-01-02,APETECH AP1935,AC0581579 FREN BALATASI ON MASTER III-MOVANO 2...,1,0,1
2,120-0000,2023-01-02,AYD 60119BA,60119A FREN BALATASI ON TRAFIC III TALENTO 1.6...,1,0,2
3,120-0000,2023-01-02,EURO EF295348,HAVA FILTRESI M9T 2.3 MASTER III-MOVANO,1,0,3
4,120-0000,2023-01-02,ORJ 152093920R,YAG FILTRESI M9T LAGUNA II-III-LATITUDE-MEGANE...,1,0,4
...,...,...,...,...,...,...,...
386400,329-0833,2023-12-08,SILBAK SB2416C,SILECEK SUPURGESI 600MMX400MM MUZ TIPI C3 PICA...,2,2014,1630
386401,329-0833,2023-12-08,STAHL EO6012,084527 MOTOR YAGI-5 LT- SUPREME PLUS 5W30 PART...,1,2014,14
386402,329-0833,2023-12-08,UFI 2504300,0986TF0029 YAG FILTRESI CORSA C COMBO C MERIVA...,1,2014,3534
386403,329-0833,2023-12-08,VALEO 32217,10 LUK KUTU SINYAL AMPUL 5W 12V SOFIM,1,2014,5888


In [6]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

In [7]:
item_features = ddf[['CustomerCode_', 'ProductCode_', 'Quantity']]  # Assuming these are the relevant features
item_features = item_features.pivot_table(index='ProductCode_', columns='CustomerCode_', values='Quantity', fill_value=0)

# Normalize item features
item_features_normalized = item_features.apply(lambda x: (x - x.mean()) / (x.max() - x.min()), axis=1)

# Calculate item-item cosine similarity
item_similarities = cosine_similarity(item_features_normalized)

# Function to get top similar items for a given item
def get_similar_items(item_index, item_similarities, top_n=5):
    similar_indices = item_similarities[item_index].argsort()[::-1]
    similar_items = [(index, item_similarities[item_index][index]) for index in similar_indices[1:top_n+1]]
    return similar_items

# Example: Recommend items similar to item with index 0
item_index = 0
similar_items = get_similar_items(item_index, item_similarities)

# Printing recommended items
print("Top 5 items similar to item with index", item_index, ":")
for item in similar_items:
    print("Item index:", item[0], "| Similarity score:", item[1])

Top 5 items similar to item with index 0 :
Item index: 18943 | Similarity score: 0.44809837812500575
Item index: 1558 | Similarity score: 0.4463761143047988
Item index: 18942 | Similarity score: 0.4386105598367809
Item index: 1 | Similarity score: 0.4222943099296743
Item index: 8331 | Similarity score: 0.3848122546207641


In [10]:
item_index=300
similar_items = get_similar_items(item_index, item_similarities)
# Printing recommended items
print("Top 5 items similar to item with index", item_index, ":")
for item in similar_items:
    print("Item index:", item[0], "| Similarity score:", item[1])

Top 5 items similar to item with index 300 :
Item index: 36861 | Similarity score: 0.7305765478215588
Item index: 14448 | Similarity score: 0.5156961350821387
Item index: 14449 | Similarity score: 0.5156961350821387
Item index: 29315 | Similarity score: 0.5156961350821357
Item index: 10971 | Similarity score: 0.4000080498894339


In [11]:
print(ddf.loc[ddf['ProductCode_'] == item_index, ['Name', 'ProductCode_']].head(1))
print("#######################################")
for i in similar_items:
    print(ddf.loc[ddf['ProductCode_'] == i[0], ['Name', 'ProductCode_']].head(1))
    print("#######################################")

                                                  Name  ProductCode_
536  YAG FILTRESI 1.5 K9K CLIO IV-DOKKER-LODGY-SAND...           300
#######################################
                                            Name  ProductCode_
295729  MOTOR KULAGI ARKA SOL-SAG R60/61 2010-16         36861
#######################################
                              Name  ProductCode_
48041  AMORTISOR ON SAG PANDA 03->         14448
#######################################
                              Name  ProductCode_
48042  AMORTISOR ON SOL PANDA 03->         14449
#######################################
                                                     Name  ProductCode_
156981  8FK351134331 KLIMA KOMPRESORU EW TU5JP4 DV4 20...         29315
#######################################
                                                    Name  ProductCode_
29980  TRIGER KAYISI BERLING C3 C4 301 ELYSEE DV6C DV...         10971
#######################################
